In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
import glob, os, sys
from shutil import copy
from xml.etree.ElementTree import Element, SubElement, Comment
import subprocess as sp
import pandas as pd
from datetime import datetime
from osgeo import ogr
import logging

In [2]:
# Set INPUT parameters
# ruweDataDir    :: input raw survey data
# asciiDataDir   :: output folder
# workDir        :: working directory (emptied each run)
# bgShp          :: background polygon shapefile of baggervakken
# logFile        :: file used to store logs

ruweDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\survey_ruweData'
asciiDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\survey_asciiData'
workdir = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\tmp'
bg_shp = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\achtergrondShp//Achtergrond_polygonen.shp' # achtergrond_shapeObj3.shp'
bg_shx = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\achtergrondShp//Achtergrond_polygonen.shx'
bg_dbf = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\achtergrondShp//Achtergrond_polygonen.dbf'
bg_prj = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\achtergrondShp//Achtergrond_polygonen.prj'
bg_cpg = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\achtergrondShp//Achtergrond_polygonen.cpg'
logFile = r'D:\Projects\Pr\3317.20\BodempeilingScript_v3\log\log_file.out'

In [3]:
# Set path OGR/GDAL files 
# ogr2ogr        :: converts simple features data between file formats
# gdalwarp       :: image reprojection and warping utility
# gdal_rasterize :: burns vector geometries into a raster
# gdal_translate :: converts raster data between different formats
# gdalbuildvrt   :: builds a VRT from a list of datasets
# gdalinfo       :: lists information about a raster dataset

ogr2ogr = r'C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe'
gdalwarp = r'C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe'
gdal_rasterize = r'C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'
gdalbuildvrt = r'C:\Python35\Lib\site-packages\osgeo//gdalbuildvrt.exe'
gdalinfo = r'C:\Python35\Lib\site-packages\osgeo\gdalinfo.exe'

In [4]:
logger = logging.getLogger('survey2arcinfoascii')
hdlr = logging.FileHandler(logFile)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)

In [5]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def writeOGRVRT(basename, fullpath):
    top = Element('OGRVRTDataSource')
    child = SubElement(top, 'OGRVRTLayer')
    child.set('name', basename)
    sub_child_1 = SubElement(child, 'SrcDataSource')
    sub_child_1.text = fullpath
    sub_child_2 = SubElement(child, 'GeometryType')
    sub_child_2.text = 'wkbPoint'
    sub_child_3 = SubElement(child, 'LayerSRS')
    sub_child_3.text = 'EPSG:28992'
    sub_child_4 = SubElement(child, 'GeometryField')
    sub_child_4.set('encoding','PointFromColumns')
    sub_child_4.set('x','field_1')
    sub_child_4.set('y','field_2')
    sub_child_4.set('z','field_3')

    return prettify(top)

In [14]:
logger.info ('Empty tmp dir %s' % (workdir))
#filelist = glob.glob(workdir+'//*')
#for f in filelist:
#    os.remove(f)
    
folder = workdir
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)    

folder = os.path.join(workdir,'bg_sel')
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)  
        
driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(bg_shp, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    logger.error ('Could not open %s' % (bg_shp))
else:
    logger.info ('Opened %s' % (bg_shp))
    layer = dataSource.GetLayer()
    bg_shp_features = layer.GetFeatureCount()
    logger.info ('Name of layer: %s' % layer.GetDescription())
    logger.info ("Number of features in %s: %d" % (os.path.basename(bg_shp),bg_shp_features))
    
    for i in range(bg_shp_features):
        feat = layer.GetFeature(i)
        obID = feat.GetField('OBJECTID')    
    logger.info (obID)

In [15]:
for root, dirs, files in os.walk(ruweDataDir):
    for file in files:
        if file.endswith('.asc'):
            init_file = os.path.join( os.path.abspath(root), file )
            base, extension = os.path.splitext(file)
            logger.info (init_file)
            
            # 0 get timestamp
            dateISO = datetime(int('20'+base[15:17]), int(base[17:19]), int(base[19:21]))
            t = dateISO.strftime("%Y%m%d%H%M%S")
            logger.info (t)            
            
            # 1 convert survey data to CSV format
            a = os.path.join(workdir,t+'.csv')
            df = pd.read_csv(init_file, header=None)
            df.to_csv(a, header=False, index=False, sep=';')
            
            # 2 build OGRVRT from CSV file
            b = os.path.join(workdir,t+'.vrt')
            with open(b, 'w') as the_file:
                the_file.write(writeOGRVRT(t, a))
                
#             # 2.0 copy bg_shp to tmp dir
#             copy(bg_shp, os.path.join(workdir,'bg_sel'))
#             copy(bg_shx, os.path.join(workdir,'bg_sel'))
#             copy(bg_dbf, os.path.join(workdir,'bg_sel'))
#             copy(bg_prj, os.path.join(workdir,'bg_sel'))
#             copy(bg_cpg, os.path.join(workdir,'bg_sel'))
            
#             # 2.1 get  SHP from OGRVRT
#             b1 = os.path.join(workdir,'bg_sel','date_'+t+'.shp')
#             command = ogr2ogr+' -sql' ' "' 'SELECT * FROM date_'+t+'" '+b1+' '+b
#             logger.info (command)
#             norm = sp.Popen(command, stdout=sp.PIPE, shell=True)
#             norm.communicate()                        
            
#             # 2.2 ST_Contains points and bg_shp
#             b2 = os.path.join(workdir,'bg_sel')
#             b3 = 'bg_shp_selection'
#             command = ogr2ogr+' -sql' ' "' 'SELECT A.* FROM Achtergrond_polygonen A, '+'date_'+t+' B WHERE ST_Contains(A.geometry, B.geometry) GROUP BY A.objectid'+'" '+ '-dialect SQLITE '+b2+' '+b2+' -nln '+b3
#             logger.info (command)
#             norm = sp.Popen(command, stdout=sp.PIPE, shell=True)
#             norm.communicate()                        

                
            # 3 create RASTER from OGRVRT
            c = os.path.join(workdir,t+'.tif')
            command = [gdal_rasterize, '-a','field_3','-tr','1.0','1.0', '-l',t,b,c]
            logger.info (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()

            
#             for feature in range(bg_shp_features):
#                 feat = layer.GetFeature(feature)
#                 obID = feat.GetField('OBJECTID')    
#                 logger.info (obID)            
                
#                 # 4 clip point RASTER with feature achtergrond SHP
#                 d = os.path.join(workdir,'peilingGrid'+str(obID).zfill(2)+'_'+t+'.tif.vrt')
#                 command = [gdalwarp, '-srcnodata', '0', '-dstnodata', '-9999', '-overwrite','-of', 'VRT', '-crop_to_cutline', 
#                            '-cutline', bg_shp, '-cwhere', 'OBJECTID = '+str(obID), c, d]            
#                 logger.info (sp.list2cmdline(command))
#                 norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
#                 norm.communicate()
                
#                 # 5A convert feature achtergrond SHP to RASTER
#                 e = os.path.join(workdir,'peilingGrid'+str(obID).zfill(2)+'_'+t+'_bg_tmp'+'.tif')
#                 command = [gdal_rasterize, '-a', 'NoDataValu', '-a_srs', 'EPSG:28992', '-where', 'OBJECTID = '+str(obID), 
#                            '-tr', '1.0', '1.0', '-l', layer.GetDescription(), bg_shp, e]         
#                 logger.info (sp.list2cmdline(command))
#                 norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
#                 norm.communicate()
                
#                 # 5B clip achtergrond RASTER
#                 f = os.path.join(workdir,'peilingGrid'+str(obID).zfill(2)+'_'+t+'_bg'+'.tif.vrt')
#                 command = [gdalwarp, '-srcnodata', '-9999', '-dstnodata', '-9999', '-of', 'VRT', '-tr', '1.0', '1.0', 
#                            '-overwrite', '-crop_to_cutline', '-cutline', bg_shp, '-cwhere', 'OBJECTID = '+str(obID), e, f]            
#                 logger.info (sp.list2cmdline(command))
#                 norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
#                 norm.communicate()                                

#                 # 6 Build VRT data source of point RASTER and feature achtergrond RASTER                
#                 g = os.path.join(workdir,'peilingGrid'+str(obID).zfill(2)+'_'+t+'.vrt')
#                 command = [gdalbuildvrt, '-srcnodata', '-9999', g, d, f]
#                 logger.info (sp.list2cmdline(command))
#                 norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
#                 norm.communicate()

#                 # 7 Convert VRT to ArcInfoASCII
#                 h = os.path.join(asciiDataDir,'peilingGrid'+str(obID).zfill(2)+'_'+t+'.asc')
#                 command = [gdal_translate, '-of', 'AAIGrid', '-tr', '1.0', '1.0', g, h]
#                 logger.info (sp.list2cmdline(command))
#                 norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
#                 norm.communicate()
                
#                 # 8 Only keep the ArcInfoASCIIs that contains data
#                 command = gdalinfo + ' -mm ' + h + ' | find "Computed"'
#                 logger.info (command)
#                 norm = sp.Popen(command, stdout=sp.PIPE, shell=True).communicate()
#                 logger.info (norm)
#                 if len(norm[0]) == 0:                    
#                     for fl in glob.glob(h[0:-4]+'*'):
#                         os.remove(fl)